## Importing the libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json 
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import os
import re    
import nltk  
import string 
import warnings
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
import numpy as np
import cv2
import pandas_profiling

# Read tweets Data

In [8]:

tweet_json=pd.read_json('train_data.json')

In [9]:

s_t= []
s_s = []
time = []
tic= []
for col in tweet_json.records:
    s_t.append(col['stocktwit_tweet']) 
    s_s.append(col['sentiment_score'])
    time.append(pd.to_datetime(col['timestamp']))
    tic.append(col['ticker'])
tweet_json_data = pd.DataFrame({'stocktwit_Tweet':s_t, 
                          'Sentiment_score':s_s,
                          'timestamp':time,
                         'ticker':tic})




In [10]:
tweet_json_data.head(-2)

,stocktwit_Tweet,Sentiment_score,timestamp,ticker
0,$AMD going up but hesitating however chart is ...,3,2018-09-19 18:38:28+00:00,$AMD
1,@inforlong @MariaGascon Despite\nChina trade w...,3,2018-10-09 03:51:06+00:00,$CAT
2,$AVGO WTF?,2,2018-07-12 13:35:32+00:00,$AVGO
3,$PH\n New Insider Filing On: \n MULLER KLAUS P...,2,2018-07-19 03:32:50+00:00,$PH
4,$FB if it bounces tommorrow do the right thing...,3,2018-08-23 19:07:54+00:00,$FB
...,...,...,...,...
1039124,$AAPL I wonder why Guy Adami is always bearish...,2,2018-09-13 01:24:37+00:00,$AAPL
1039125,$AMD prepare yourselves for launch. Closing 28...,3,2018-09-04 18:01:16+00:00,$AMD
1039126,$CLF https://twitter.com/hrdrckminer/status/10...,2,2018-08-13 22:15:04+00:00,$CLF
1039127,$AMD 🔥🔥🔥,3,2018-09-04 15:25:59+00:00,$AMD


In [11]:
tweet_json_data.to_csv('tweet_json_data.csv',index=False)


In [12]:
tweet_json_data.head()

,stocktwit_Tweet,Sentiment_score,timestamp,ticker
0,$AMD going up but hesitating however chart is ...,3,2018-09-19 18:38:28+00:00,$AMD
1,@inforlong @MariaGascon Despite\nChina trade w...,3,2018-10-09 03:51:06+00:00,$CAT
2,$AVGO WTF?,2,2018-07-12 13:35:32+00:00,$AVGO
3,$PH\n New Insider Filing On: \n MULLER KLAUS P...,2,2018-07-19 03:32:50+00:00,$PH
4,$FB if it bounces tommorrow do the right thing...,3,2018-08-23 19:07:54+00:00,$FB


# Pre-Processing of TWEET data

In [19]:
import urllib.request as url 

#!pip install bs4
from bs4 import BeautifulSoup as bs

#!pip install wordcloud
from wordcloud import WordCloud

import re
import requests


import random
random.seed(123)
import spacy

In [20]:
#pip install emoji

### Removing the duplicate TWEETS

In [21]:
tweet_json_data['stocktwit_Tweet'].drop_duplicates(keep='first', inplace=True)


### Converting Emoji to Text

In [24]:
import emoji

tweet_json_data['cln_tweet']=tweet_json_data['stocktwit_Tweet'].apply(lambda x: emoji.demojize(x))

In [25]:
tweet_json_data

,stocktwit_Tweet,Sentiment_score,timestamp,ticker,cln_tweet
0,$AMD going up but hesitating however chart is ...,3,2018-09-19 18:38:28+00:00,$AMD,$AMD going up but hesitating however chart is ...
1,@inforlong @MariaGascon Despite\nChina trade w...,3,2018-10-09 03:51:06+00:00,$CAT,@inforlong @MariaGascon Despite\nChina trade w...
2,$AVGO WTF?,2,2018-07-12 13:35:32+00:00,$AVGO,$AVGO WTF?
3,$PH\n New Insider Filing On: \n MULLER KLAUS P...,2,2018-07-19 03:32:50+00:00,$PH,$PH\n New Insider Filing On: \n MULLER KLAUS P...
4,$FB if it bounces tommorrow do the right thing...,3,2018-08-23 19:07:54+00:00,$FB,$FB if it bounces tommorrow do the right thing...
...,...,...,...,...,...
1039126,$CLF https://twitter.com/hrdrckminer/status/10...,2,2018-08-13 22:15:04+00:00,$CLF,$CLF https://twitter.com/hrdrckminer/status/10...
1039127,$AMD 🔥🔥🔥,3,2018-09-04 15:25:59+00:00,$AMD,NaN
1039128,$FB damn should of shorted 196 after hours to ...,0,2018-07-25 21:29:03+00:00,$FB,$FB damn should of shorted 196 after hours to ...
1039129,$AMD back to the 31&#39;s,0,2018-09-28 16:19:55+00:00,$AMD,$AMD back to the 31&#39;s


### Removing the url's from tweets

In [27]:
tweet_json_data['cln_tweet'] = tweet_json_data['cln_tweet'].str.replace('http\S*|www.\S*','', case=False)
tweet_json_data

,stocktwit_Tweet,Sentiment_score,timestamp,ticker,cln_tweet
0,$AMD going up but hesitating however chart is ...,3,2018-09-19 18:38:28+00:00,$AMD,$AMD going up but hesitating however chart is ...
1,@inforlong @MariaGascon Despite\nChina trade w...,3,2018-10-09 03:51:06+00:00,$CAT,@inforlong @MariaGascon Despite\nChina trade w...
2,$AVGO WTF?,2,2018-07-12 13:35:32+00:00,$AVGO,$AVGO WTF?
3,$PH\n New Insider Filing On: \n MULLER KLAUS P...,2,2018-07-19 03:32:50+00:00,$PH,$PH\n New Insider Filing On: \n MULLER KLAUS P...
4,$FB if it bounces tommorrow do the right thing...,3,2018-08-23 19:07:54+00:00,$FB,$FB if it bounces tommorrow do the right thing...
...,...,...,...,...,...
1039126,$CLF https://twitter.com/hrdrckminer/status/10...,2,2018-08-13 22:15:04+00:00,$CLF,$CLF
1039127,$AMD 🔥🔥🔥,3,2018-09-04 15:25:59+00:00,$AMD,NaN
1039128,$FB damn should of shorted 196 after hours to ...,0,2018-07-25 21:29:03+00:00,$FB,$FB damn should of shorted 196 after hours to ...
1039129,$AMD back to the 31&#39;s,0,2018-09-28 16:19:55+00:00,$AMD,$AMD back to the 31&#39;s


### Removing the @mentions count

In [29]:
tweet_json_data['cln_tweet'] = tweet_json_data['cln_tweet'].str.replace("@[\w]*","")

tweet_json_data.head(5)

,stocktwit_Tweet,Sentiment_score,timestamp,ticker,cln_tweet
0,$AMD going up but hesitating however chart is ...,3,2018-09-19 18:38:28+00:00,$AMD,$AMD going up but hesitating however chart is ...
1,@inforlong @MariaGascon Despite\nChina trade w...,3,2018-10-09 03:51:06+00:00,$CAT,Despite\nChina trade war $CAT held very well...
2,$AVGO WTF?,2,2018-07-12 13:35:32+00:00,$AVGO,$AVGO WTF?
3,$PH\n New Insider Filing On: \n MULLER KLAUS P...,2,2018-07-19 03:32:50+00:00,$PH,$PH\n New Insider Filing On: \n MULLER KLAUS P...
4,$FB if it bounces tommorrow do the right thing...,3,2018-08-23 19:07:54+00:00,$FB,$FB if it bounces tommorrow do the right thing...


In [30]:
tweet_json_data.shape

(1039131, 5)

## Empty tweets

In [33]:
emptytweets1 = tweet_json_data[['stocktwit_Tweet','ticker']].apply(lambda row: (len(row['stocktwit_Tweet'])-len(row['ticker']) ==0), axis=1)


emptytweets = tweet_json_data[emptytweets1]

In [34]:
emptytweets.head()

,stocktwit_Tweet,Sentiment_score,timestamp,ticker,cln_tweet
13,$AAPL,2,2018-10-06 21:38:04+00:00,$AAPL,$AAPL
14,$ANTM,2,2018-08-08 13:34:42+00:00,$ANTM,$ANTM
239,$NVDA,2,2018-08-22 14:57:24+00:00,$NVDA,$NVDA
260,$NFLX,2,2018-08-16 12:31:41+00:00,$NFLX,$NFLX
268,$AMD,3,2018-09-12 16:57:43+00:00,$AMD,$AMD


In [35]:
emptytweets.shape

(11370, 5)

### Removing empty tweets

In [39]:
tweet_json_data=tweet_json_data[-emptytweets1]

In [40]:
tweet_json_data.head()

,stocktwit_Tweet,Sentiment_score,timestamp,ticker,cln_tweet
0,$AMD going up but hesitating however chart is ...,3,2018-09-19 18:38:28+00:00,$AMD,$AMD going up but hesitating however chart is ...
1,@inforlong @MariaGascon Despite\nChina trade w...,3,2018-10-09 03:51:06+00:00,$CAT,Despite\nChina trade war $CAT held very well...
2,$AVGO WTF?,2,2018-07-12 13:35:32+00:00,$AVGO,$AVGO WTF?
3,$PH\n New Insider Filing On: \n MULLER KLAUS P...,2,2018-07-19 03:32:50+00:00,$PH,$PH\n New Insider Filing On: \n MULLER KLAUS P...
4,$FB if it bounces tommorrow do the right thing...,3,2018-08-23 19:07:54+00:00,$FB,$FB if it bounces tommorrow do the right thing...


In [41]:
tweet_json_data.shape

(1027761, 5)

### Removing  Numbers,Special Characters & Hashtags

In [44]:
tweet_json_data['cln_tweet']=tweet_json_data['cln_tweet'].str.replace('\$[\w]*','')

In [45]:
tweet_json_data['cln_tweet'] = tweet_json_data['cln_tweet'].str.replace("[^a-zA-Z]", " ") 
tweet_json_data.head(8)

,stocktwit_Tweet,Sentiment_score,timestamp,ticker,cln_tweet
0,$AMD going up but hesitating however chart is ...,3,2018-09-19 18:38:28+00:00,$AMD,going up but hesitating however chart is very...
1,@inforlong @MariaGascon Despite\nChina trade w...,3,2018-10-09 03:51:06+00:00,$CAT,Despite China trade war held very well thu...
2,$AVGO WTF?,2,2018-07-12 13:35:32+00:00,$AVGO,WTF
3,$PH\n New Insider Filing On: \n MULLER KLAUS P...,2,2018-07-19 03:32:50+00:00,$PH,New Insider Filing On MULLER KLAUS PETER ...
4,$FB if it bounces tommorrow do the right thing...,3,2018-08-23 19:07:54+00:00,$FB,if it bounces tommorrow do the right thing an...
5,$FB as usual......RED \nevery day.,0,2018-08-02 11:40:49+00:00,$FB,as usual RED every day
6,$AET UnitedHealth Group Goes Ex-Dividend Today...,2,2018-09-07 15:01:06+00:00,$AET,UnitedHealth Group Goes Ex Dividend Today
7,$NOC let&#39;s see when we can hit 310,4,2018-09-12 20:48:40+00:00,$NOC,let s see when we can hit


### Removing the Dollar sign and converting TICKER column to uppercase

In [48]:
tweet_json_data['ticker']=tweet_json_data['ticker'].str.replace('$','')

In [49]:
tweet_json_data['ticker']=tweet_json_data['ticker'].str.upper()

In [50]:
tweet_json_data.head()

,stocktwit_Tweet,Sentiment_score,timestamp,ticker,cln_tweet
0,$AMD going up but hesitating however chart is ...,3,2018-09-19 18:38:28+00:00,AMD,going up but hesitating however chart is very...
1,@inforlong @MariaGascon Despite\nChina trade w...,3,2018-10-09 03:51:06+00:00,CAT,Despite China trade war held very well thu...
2,$AVGO WTF?,2,2018-07-12 13:35:32+00:00,AVGO,WTF
3,$PH\n New Insider Filing On: \n MULLER KLAUS P...,2,2018-07-19 03:32:50+00:00,PH,New Insider Filing On MULLER KLAUS PETER ...
4,$FB if it bounces tommorrow do the right thing...,3,2018-08-23 19:07:54+00:00,FB,if it bounces tommorrow do the right thing an...


### Removing repeating words

In [55]:
tweet_json_data['cln_tweet'] = tweet_json_data['cln_tweet'].apply(lambda x : re.sub(r'(.)\1{1,}', r'\1\1', x) )

### Expanding contractions

In [59]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [60]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

In [61]:
tweet_json_data['cln_tweet'] = tweet_json_data['cln_tweet'].apply(expand_contractions)
tweet_json_data.head()

,stocktwit_Tweet,Sentiment_score,timestamp,ticker,cln_tweet
0,$AMD going up but hesitating however chart is ...,3,2018-09-19 18:38:28+00:00,AMD,going up but hesitating however chart is very...
1,@inforlong @MariaGascon Despite\nChina trade w...,3,2018-10-09 03:51:06+00:00,CAT,Despite China trade war held very well thu...
2,$AVGO WTF?,2,2018-07-12 13:35:32+00:00,AVGO,WTF
3,$PH\n New Insider Filing On: \n MULLER KLAUS P...,2,2018-07-19 03:32:50+00:00,PH,New Insider Filing On MULLER KLAUS PETER Tr...
4,$FB if it bounces tommorrow do the right thing...,3,2018-08-23 19:07:54+00:00,FB,if it bounces tommorrow do the right thing an...


### Converting to lower case

In [63]:
tweet_json_data['cln_tweet']=tweet_json_data['cln_tweet'].str.lower()
tweet_json_data.head()

,stocktwit_Tweet,Sentiment_score,timestamp,ticker,cln_tweet
0,$AMD going up but hesitating however chart is ...,3,2018-09-19 18:38:28+00:00,AMD,going up but hesitating however chart is very...
1,@inforlong @MariaGascon Despite\nChina trade w...,3,2018-10-09 03:51:06+00:00,CAT,despite china trade war held very well thu...
2,$AVGO WTF?,2,2018-07-12 13:35:32+00:00,AVGO,wtf
3,$PH\n New Insider Filing On: \n MULLER KLAUS P...,2,2018-07-19 03:32:50+00:00,PH,new insider filing on muller klaus peter tr...
4,$FB if it bounces tommorrow do the right thing...,3,2018-08-23 19:07:54+00:00,FB,if it bounces tommorrow do the right thing an...


### Time  removing from Timestamp

In [66]:
tweet_json_data['timestamp'] = tweet_json_data['timestamp'].astype('str')

In [67]:
tweet_json_data

,stocktwit_Tweet,Sentiment_score,timestamp,ticker,cln_tweet
0,$AMD going up but hesitating however chart is ...,3,2018-09-19 18:38:28+00:00,AMD,going up but hesitating however chart is very...
1,@inforlong @MariaGascon Despite\nChina trade w...,3,2018-10-09 03:51:06+00:00,CAT,despite china trade war held very well thu...
2,$AVGO WTF?,2,2018-07-12 13:35:32+00:00,AVGO,wtf
3,$PH\n New Insider Filing On: \n MULLER KLAUS P...,2,2018-07-19 03:32:50+00:00,PH,new insider filing on muller klaus peter tr...
4,$FB if it bounces tommorrow do the right thing...,3,2018-08-23 19:07:54+00:00,FB,if it bounces tommorrow do the right thing an...
...,...,...,...,...,...
1039126,$CLF https://twitter.com/hrdrckminer/status/10...,2,2018-08-13 22:15:04+00:00,CLF,
1039127,$AMD 🔥🔥🔥,3,2018-09-04 15:25:59+00:00,AMD,nan
1039128,$FB damn should of shorted 196 after hours to ...,0,2018-07-25 21:29:03+00:00,FB,damn should of shorted after hours to late now
1039129,$AMD back to the 31&#39;s,0,2018-09-28 16:19:55+00:00,AMD,back to the s


In [68]:
tweet_json_data['timestamp'] = tweet_json_data['timestamp'].str.split(' ').str[0]

In [69]:
tweet_json_data['month'] = tweet_json_data['timestamp'].str.split('-', expand = True)[1]
tweet_json_data['day'] = tweet_json_data['timestamp'].str.split('-', expand = True)[2]

In [70]:
tweet_json_data.head()

,stocktwit_Tweet,Sentiment_score,timestamp,ticker,cln_tweet,month,day
0,$AMD going up but hesitating however chart is ...,3,2018-09-19,AMD,going up but hesitating however chart is very...,09,19
1,@inforlong @MariaGascon Despite\nChina trade w...,3,2018-10-09,CAT,despite china trade war held very well thu...,10,09
2,$AVGO WTF?,2,2018-07-12,AVGO,wtf,07,12
3,$PH\n New Insider Filing On: \n MULLER KLAUS P...,2,2018-07-19,PH,new insider filing on muller klaus peter tr...,07,19
4,$FB if it bounces tommorrow do the right thing...,3,2018-08-23,FB,if it bounces tommorrow do the right thing an...,08,23


#### Removing the untidy tweet column


In [75]:
tweet_json_data.drop('stocktwit_Tweet', axis=1,inplace=True)

In [76]:
tweet_json_data.head()

,Sentiment_score,timestamp,ticker,cln_tweet,month,day
0,3,2018-09-19,AMD,going up but hesitating however chart is very...,09,19
1,3,2018-10-09,CAT,despite china trade war held very well thu...,10,09
2,2,2018-07-12,AVGO,wtf,07,12
3,2,2018-07-19,PH,new insider filing on muller klaus peter tr...,07,19
4,3,2018-08-23,FB,if it bounces tommorrow do the right thing an...,08,23


In [77]:
tweet_json_data['month'].value_counts()

10    271686
08    259302
07    251086
09    245687
Name: month, dtype: int64

### GroupBy with Timestamp and Ticket- Aggregating the sentiment score

In [89]:
tweet_json_data_grpdate=tweet_json_data.groupby(by=['timestamp','ticker'])['Sentiment_score'].agg(lambda x:x.value_counts().index[0]).to_frame()



In [90]:
tweet_json_data_grpdate.shape

(57566, 1)

In [91]:
tweet_json_data_grpdate.head()

Sentiment_score
timestamp  ticker                 
2018-07-01 AABA                  2
           AAL                   3
           AAP                   1
           AAPL                  2
           ABBV                  0

In [94]:
#tweet_json_data_grpdate=tweet_json_data_grpdate['Sentiment_score']

In [260]:
tweet_json_data.columns

Index(['Sentiment_score', 'timestamp', 'ticker', 'cln_tweet', 'month', 'day'], dtype='object')

In [261]:
tweet_json_model_df = tweet_json_data.filter(['Sentiment_score', 'timestamp', 'ticker', 'cln_tweet', 'month', 'day'], axis=1)

In [262]:
#tweet_json_model_df= tweet_json_data

In [263]:
tweet_json_model_df.shape

(1027761, 6)

In [264]:
tweet_json_model_df.head()

,Sentiment_score,timestamp,ticker,cln_tweet,month,day
0,3,2018-09-19,AMD,going up but hesitating however chart is very...,09,19
1,3,2018-10-09,CAT,despite china trade war held very well thu...,10,09
2,2,2018-07-12,AVGO,wtf,07,12
3,2,2018-07-19,PH,new insider filing on muller klaus peter tr...,07,19
4,3,2018-08-23,FB,if it bounces tommorrow do the right thing an...,08,23


### Tokenization

In [265]:
from nltk.corpus import wordnet
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

from nltk.tokenize.toktok import ToktokTokenizer
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import en_core_web_sm
from sklearn.metrics import classification_report,confusion_matrix

In [266]:
tweet_json_model_df['token_tweets'] = tweet_json_model_df['cln_tweet'].apply(lambda x: x.split()) # tokenizing

### Stop words removal

In [267]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [268]:
from tqdm import tqdm_notebook as tq

stop_words_removed = []

for tokenized_words in tq(tweet_json_model_df['token_tweets']):
    
    stop_words_removed.append([word for word in tokenized_words if word not in stop])
    


In [269]:
tweet_json_model_df['sw_rem_data'] = stop_words_removed

## Lemmatization

In [270]:
lemmatizer = WordNetLemmatizer()
tweet_json_model_df['lemm_words'] = tweet_json_model_df['sw_rem_data'].apply(lambda x: [lemmatizer.lemmatize(i,pos = 'v') for i in x])



In [271]:
tweet_json_model_df

,Sentiment_score,timestamp,ticker,cln_tweet,month,day,token_tweets,sw_rem_data,lemm_words
0,3,2018-09-19,AMD,going up but hesitating however chart is very...,09,19,"[going, up, but, hesitating, however, chart, i...","[going, hesitating, however, chart, stable, go...","[go, hesitate, however, chart, stable, go, upw..."
1,3,2018-10-09,CAT,despite china trade war held very well thu...,10,09,"[despite, china, trade, war, held, very, well,...","[despite, china, trade, war, held, well, thumbs]","[despite, china, trade, war, hold, well, thumb]"
2,2,2018-07-12,AVGO,wtf,07,12,[wtf],[wtf],[wtf]
3,2,2018-07-19,PH,new insider filing on muller klaus peter tr...,07,19,"[new, insider, filing, on, muller, klaus, pete...","[new, insider, filing, muller, klaus, peter, t...","[new, insider, file, muller, klaus, peter, tra..."
4,3,2018-08-23,FB,if it bounces tommorrow do the right thing an...,08,23,"[if, it, bounces, tommorrow, do, the, right, t...","[bounces, tommorrow, right, thing, gtfo]","[bounce, tommorrow, right, thing, gtfo]"
...,...,...,...,...,...,...,...,...,...
1039126,2,2018-08-13,CLF,,08,13,[],[],[]
1039127,3,2018-09-04,AMD,nan,09,04,[nan],[nan],[nan]
1039128,0,2018-07-25,FB,damn should of shorted after hours to late now,07,25,"[damn, should, of, shorted, after, hours, to, ...","[damn, shorted, hours, late]","[damn, short, hours, late]"
1039129,0,2018-09-28,AMD,back to the s,09,28,"[back, to, the, s]",[back],[back]


In [272]:
tweet_json_model_df['cleanedTweet'] = tweet_json_model_df['lemm_words'].apply(lambda x: ' '.join(x))

In [273]:
tweet_json_model_df

,Sentiment_score,timestamp,ticker,cln_tweet,month,day,token_tweets,sw_rem_data,lemm_words,cleanedTweet
0,3,2018-09-19,AMD,going up but hesitating however chart is very...,09,19,"[going, up, but, hesitating, however, chart, i...","[going, hesitating, however, chart, stable, go...","[go, hesitate, however, chart, stable, go, upw...",go hesitate however chart stable go upward
1,3,2018-10-09,CAT,despite china trade war held very well thu...,10,09,"[despite, china, trade, war, held, very, well,...","[despite, china, trade, war, held, well, thumbs]","[despite, china, trade, war, hold, well, thumb]",despite china trade war hold well thumb
2,2,2018-07-12,AVGO,wtf,07,12,[wtf],[wtf],[wtf],wtf
3,2,2018-07-19,PH,new insider filing on muller klaus peter tr...,07,19,"[new, insider, filing, on, muller, klaus, pete...","[new, insider, filing, muller, klaus, peter, t...","[new, insider, file, muller, klaus, peter, tra...",new insider file muller klaus peter transactio...
4,3,2018-08-23,FB,if it bounces tommorrow do the right thing an...,08,23,"[if, it, bounces, tommorrow, do, the, right, t...","[bounces, tommorrow, right, thing, gtfo]","[bounce, tommorrow, right, thing, gtfo]",bounce tommorrow right thing gtfo
...,...,...,...,...,...,...,...,...,...,...
1039126,2,2018-08-13,CLF,,08,13,[],[],[],
1039127,3,2018-09-04,AMD,nan,09,04,[nan],[nan],[nan],nan
1039128,0,2018-07-25,FB,damn should of shorted after hours to late now,07,25,"[damn, should, of, shorted, after, hours, to, ...","[damn, shorted, hours, late]","[damn, short, hours, late]",damn short hours late
1039129,0,2018-09-28,AMD,back to the s,09,28,"[back, to, the, s]",[back],[back],back


In [274]:
#tweet_json_model_df=tweet_json_model_df.drop("cleanTweet", axis=1)
tweet_json_model_df=tweet_json_model_df.drop("token_tweets", axis=1)
tweet_json_model_df=tweet_json_model_df.drop("sw_rem_data", axis=1)

In [275]:
tweet_json_model_df=tweet_json_model_df.drop("lemm_words", axis=1)
tweet_json_model_df=tweet_json_model_df.drop("cln_tweet", axis=1)

In [276]:
tweet_json_model_df

,Sentiment_score,timestamp,ticker,month,day,cleanedTweet
0,3,2018-09-19,AMD,09,19,go hesitate however chart stable go upward
1,3,2018-10-09,CAT,10,09,despite china trade war hold well thumb
2,2,2018-07-12,AVGO,07,12,wtf
3,2,2018-07-19,PH,07,19,new insider file muller klaus peter transactio...
4,3,2018-08-23,FB,08,23,bounce tommorrow right thing gtfo
...,...,...,...,...,...,...
1039126,2,2018-08-13,CLF,08,13,
1039127,3,2018-09-04,AMD,09,04,nan
1039128,0,2018-07-25,FB,07,25,damn short hours late
1039129,0,2018-09-28,AMD,09,28,back


In [277]:
tweet_df=tweet_json_model_df['cleanedTweet'] # Collecting the tweets for model building

In [278]:
tweet_train_df=tweet_json_model_df['cleanedTweet'] # Collecting the tweets for model building (for tweet_train experiment)


In [280]:
tweet_sc=tweet_json_model_df['Sentiment_score']

In [281]:
tweet_df.shape

(1027761,)

In [282]:
tweet_train_df.shape

(1027761,)

In [283]:
tweet_sc.shape

(1027761,)

In [284]:
tweet_json_model_df

,Sentiment_score,timestamp,ticker,month,day,cleanedTweet
0,3,2018-09-19,AMD,09,19,go hesitate however chart stable go upward
1,3,2018-10-09,CAT,10,09,despite china trade war hold well thumb
2,2,2018-07-12,AVGO,07,12,wtf
3,2,2018-07-19,PH,07,19,new insider file muller klaus peter transactio...
4,3,2018-08-23,FB,08,23,bounce tommorrow right thing gtfo
...,...,...,...,...,...,...
1039126,2,2018-08-13,CLF,08,13,
1039127,3,2018-09-04,AMD,09,04,nan
1039128,0,2018-07-25,FB,07,25,damn short hours late
1039129,0,2018-09-28,AMD,09,28,back


## Train test split for building 

In [290]:
x_train_tw, x_test_tw, y_train_tw, y_test_tw = train_test_split(tweet_df, tweet_sc, test_size = 0.3, random_state=123, stratify=tweet_sc)



In [292]:
print(x_train_tw.shape)
print(x_test_tw.shape)
print(y_train_tw.shape)
print(y_test_tw.shape)

(719432,)
(308329,)
(719432,)
(308329,)


In [295]:
from sklearn.feature_extraction.text import TfidfVectorizer



In [296]:
tfidf = TfidfVectorizer() # for normal train tweets

In [297]:
tdm_x_train = tfidf.fit_transform(x_train_tw)
tdm_x_test = tfidf.transform(x_test_tw)

In [299]:
from scipy import sparse
from scipy.sparse import csr_matrix

x_train_tw_v = sparse.csr_matrix(tdm_x_train)
x_test_tw_v = sparse.csr_matrix(tdm_x_test)

In [302]:
tdm_x_train

<719432x58626 sparse matrix of type '<class 'numpy.float64'>'
	with 4204309 stored elements in Compressed Sparse Row format>

In [306]:
print(x_train_tw_v.shape)
x_test_tw_v.shape

(719432, 58626)


(308329, 58626)

# Logistic regression for Sentiment Analysis

In [308]:
logreg = LogisticRegression()
logreg.fit(x_train_tw_v,y_train_tw)

# Predictions on train data
y_pred_train_tw=logreg.predict(x_train_tw_v)
# Train data confusion Matrix
print("confusion_matrix on train data\n",confusion_matrix(y_train_tw,y_pred_train_tw))

# Predictions on test data
y_pred_test_tw=logreg.predict(x_test_tw_v)
# Test data confusion Matrix
print("confusion_matrix on test data\n",confusion_matrix(y_test_tw,y_pred_test_tw))

confusion_matrix on train data
 [[ 30350   6316  21338   2890   3521]
 [  4291  27536  38065   9577   4603]
 [  5282   5933 281966  16960   9127]
 [  4695   7361  62817  55035  14035]
 [  1133   1220  28908  17274  59199]]
confusion_matrix on test data
 [[ 12820   2797   9207   1216   1567]
 [  1881  11555  16305   4241   2049]
 [  2327   2645 120182   7484   4192]
 [  2011   3304  27052  22970   6353]
 [   495    551  12427   7586  25112]]


In [309]:
print("\nClassification Report on Train Data\n",classification_report(y_train_tw,y_pred_train_tw,digits=2))
print("\nClassification Report on Test Data\n",classification_report(y_test_tw,y_pred_test_tw,digits=2))


Classification Report on Train Data
               precision    recall  f1-score   support

           0       0.66      0.47      0.55     64415
           1       0.57      0.33      0.42     84072
           2       0.65      0.88      0.75    319268
           3       0.54      0.38      0.45    143943
           4       0.65      0.55      0.60    107734

    accuracy                           0.63    719432
   macro avg       0.62      0.52      0.55    719432
weighted avg       0.62      0.63      0.61    719432


Classification Report on Test Data
               precision    recall  f1-score   support

           0       0.66      0.46      0.54     27607
           1       0.55      0.32      0.41     36031
           2       0.65      0.88      0.75    136830
           3       0.53      0.37      0.44     61690
           4       0.64      0.54      0.59     46171

    accuracy                           0.62    308329
   macro avg       0.61      0.52      0.54    308329
we

## Naive Baye's for sentiment score

In [310]:
classifier = MultinomialNB()
classifier.fit(x_train_tw_v,y_train_tw)

# Predictions on train data
y_pred_train_tw_nb=classifier.predict(x_train_tw_v)
print("confusion_matrix on train data\n",confusion_matrix(y_train_tw,y_pred_train_tw_nb))

# Predictions on test data
y_pred_test_tw_nb=classifier.predict(x_test_tw_v)
print("confusion_matrix on test data\n",confusion_matrix(y_test_tw,y_pred_test_tw_nb))

confusion_matrix on train data
 [[ 17095   1818  36974   5518   3010]
 [  1196  16608  52556  10339   3373]
 [  1215   1347 294829  14559   7318]
 [   939   3206  83396  46090  10312]
 [   270    436  46948  18245  41835]]
confusion_matrix on test data
 [[  6787    836  16261   2373   1350]
 [   515   6496  23111   4421   1488]
 [   541    675 125231   6998   3385]
 [   386   1463  37771  17290   4780]
 [   120    169  20957   8337  16588]]


In [311]:
print("Classification Report on Train Data\n",classification_report(y_train_tw,y_pred_train_tw_nb,digits=2))
print("Classification Report on Test Data\n",classification_report(y_test_tw,y_pred_test_tw_nb,digits=2))

Classification Report on Train Data
               precision    recall  f1-score   support

           0       0.83      0.27      0.40     64415
           1       0.71      0.20      0.31     84072
           2       0.57      0.92      0.71    319268
           3       0.49      0.32      0.39    143943
           4       0.64      0.39      0.48    107734

    accuracy                           0.58    719432
   macro avg       0.65      0.42      0.46    719432
weighted avg       0.60      0.58      0.54    719432

Classification Report on Test Data
               precision    recall  f1-score   support

           0       0.81      0.25      0.38     27607
           1       0.67      0.18      0.28     36031
           2       0.56      0.92      0.70    136830
           3       0.44      0.28      0.34     61690
           4       0.60      0.36      0.45     46171

    accuracy                           0.56    308329
   macro avg       0.62      0.40      0.43    308329
weig

In [312]:
x_train_tw_v.shape

(719432, 58626)

## Neural network for Sentiment analysis

In [313]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D,Dense, Flatten,Dropout,MaxPool2D
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import os
import numpy as np
import pandas as pd

# Importing Functions from Sklearn Module
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as skm

# Module for Neural Network Model Building - Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras import optimizers

# Visualization Modules
import matplotlib.pyplot as plt
%matplotlib inline  
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, accuracy_score

In [314]:

from keras.utils import to_categorical
y_binary2 = to_categorical(y_train_tw)

In [315]:
y_train_tw.shape

(719432,)

In [316]:
y_binary2

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [317]:

dl_model_nn_tw= Sequential()

In [318]:


dl_model_nn_tw.add(Dense(5, input_dim=58626, activation='relu',  kernel_initializer='glorot_normal'))
dl_model_nn_tw.add(Dense(25, activation='relu', kernel_initializer='glorot_normal'))
dl_model_nn_tw.add(Dense(50, activation='relu', kernel_initializer='glorot_normal'))
dl_model_nn_tw.add(Dense(150, activation='relu', kernel_initializer='glorot_normal'))
dl_model_nn_tw.add(Dense(200, activation='relu', kernel_initializer='glorot_normal'))
dl_model_nn_tw.add(Dense(150, activation='relu', kernel_initializer='glorot_normal'))
dl_model_nn_tw.add(Dense(100, activation='relu', kernel_initializer='glorot_normal'))
dl_model_nn_tw.add(Dense(50, activation='relu', kernel_initializer='glorot_normal'))
dl_model_nn_tw.add(Dense(units=5, activation='softmax', kernel_initializer='glorot_normal'))



In [319]:
dl_model_nn_tw.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
dl_model_nn_tw.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 5)                 293135    
_________________________________________________________________
dense_7 (Dense)              (None, 25)                150       
_________________________________________________________________
dense_8 (Dense)              (None, 50)                1300      
_________________________________________________________________
dense_9 (Dense)              (None, 150)               7650      
_________________________________________________________________
dense_10 (Dense)             (None, 200)               30200     
_________________________________________________________________
dense_11 (Dense)             (None, 150)               30150     
_________________________________________________________________
dense_12 (Dense)             (None, 100)              

In [320]:
dl_model_nn_tw.fit(x=x_train_tw_v, y=y_binary2, batch_size=32, epochs=5, validation_split=0.2)

Train on 575545 samples, validate on 143887 samples
Epoch 1/5
575545/575545 [==============================] - 355s 616us/step - loss: 1.0258 - accuracy: 0.6153 - val_loss: 0.9579 - val_accuracy: 0.6491
Epoch 2/5
575545/575545 [==============================] - 384s 667us/step - loss: 0.9225 - accuracy: 0.6574 - val_loss: 0.9325 - val_accuracy: 0.6546
Epoch 3/5
575545/575545 [==============================] - 354s 615us/step - loss: 0.8969 - accuracy: 0.6642 - val_loss: 0.9337 - val_accuracy: 0.6522
Epoch 4/5
575545/575545 [==============================] - 356s 618us/step - loss: 0.8811 - accuracy: 0.6693 - val_loss: 0.9351 - val_accuracy: 0.6537
Epoch 5/5
575545/575545 [==============================] - 372s 646us/step - loss: 0.8709 - accuracy: 0.6734 - val_loss: 0.9309 - val_accuracy: 0.6537


In [321]:
y_train_pred_nn_tw=dl_model_nn_tw.predict_classes(x_train_tw_v)
y_test_pred_nn_tw=dl_model_nn_tw.predict_classes(x_test_tw_v)
confusion_matrix(y_true=y_test_tw, y_pred=y_test_pred_nn_tw)


array([[ 14166,   4124,   6115,   1078,   2124],
       [  1431,  15445,  12695,   3967,   2493],
       [  2714,   3368, 116869,   8186,   5693],
       [  2295,   4071,  23448,  25456,   6420],
       [   550,    395,   7914,   7672,  29640]])

In [322]:
##### Train Accuracy

accuracy_train = accuracy_score(y_train_tw, y_train_pred_nn_tw)
print('Train_Accuracy: {:.2f}'.format(accuracy_train))
f1=f1_score(y_train_tw, y_train_pred_nn_tw, average='macro')
print('Train F1-Score: {:.2f}'.format(f1))


##### Test Accuracy

accuracy_test = accuracy_score(y_test_tw, y_test_pred_nn_tw)
print('Test_Accuracy: {:.2f}'.format(accuracy_test))
f1=f1_score(y_test_tw, y_test_pred_nn_tw, average='macro')
print('Validation F1-Score: {:.2f}'.format(f1))

Train_Accuracy: 0.68
Train F1-Score: 0.62
Test_Accuracy: 0.65
Validation F1-Score: 0.59
